In [2]:
import sys
import os
sys.path.insert(1, '../scripts/')
import xarray
import yaml
import numpy as np
import pandas
import pathlib
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
# load functions from my scripts file "ssm_utils"
from ssm_utils import get_nearest_node, reshape_fvcom3D, calc_fvcom_stat, extract_fvcom_level

In [3]:
output_base = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/data')
graphics_directory = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/graphics')

model_var = 'DOXG'
output_dir = output_base/model_var

In [4]:
# create output directories if they don't exist for (1) all depths
if os.path.exists(output_dir)==False:
    print(f'creating: {output_dir}')
    os.makedirs(output_dir)
# bottom depth only
if os.path.exists(output_dir/'bottom')==False:
    print(f'creating: {output_dir}/bottom')
    os.makedirs(output_dir/'bottom')

In [5]:
with open('../etc/SSM_config.yaml', 'r') as file:
    ssm = yaml.safe_load(file)
# get shapefile path    
shp = ssm['shapefile_path']
# load shapefile into geopandas dataframe
gdf = gpd.read_file(shp)
gdf.head(1)

,node_id,Basin,uncategori,x,y,x_shp,y_shp,lat,lon,depth,included_i,DO_std,volume,area,Shape_Leng,Shape_Area,geometry
0,1249,SJF_Admiralty,None,380473.98,5365288.5,380500.860294,5.365280e+06,48.429395,-124.615801,160.628998,1,7,1.824350e+09,11357538.53,13249.778795,1.135754e+07,"POLYGON ((378570.860 5364434.875, 378213.530 5..."


In [8]:
ssm['output_paths']

['/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/wqm_baseline/ssm_output.nc',
 '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/1b_all_sog_wwtp_off/ssm_output.nc',
 '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/1c_all_sog_riv_off/ssm_output.nc',
 '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/1d_small_sog_wwtp_off/ssm_output.nc',
 '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/1e_med_sog_wwtp_off/ssm_output.nc',
 '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/2a_sog_river_0.5times/ssm_output.nc',
 '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/2b_sog_river_2times/ssm_output.nc']

In [6]:
[*ssm['run_tag']]

['0p5x_SOGNB_RiverLoading',
 '2p0x_SOGNB_RiverLoading',
 'Baseline',
 'No_SOGNB_RiverLoading',
 'No_SOGNB_WWTP',
 'No_gt100_WWTP',
 'No_lt100_WWTP']

In [ ]:
%%time
# loop through comparison cases and get timeseries from model output
#for index,run_type in enumerate(ssm['run_tag']):
for index,run_type in enumerate([*ssm['run_tag']][3]):
    # input netcdf filename
    path=pathlib.Path(
        ssm['output_paths'][index]
    )
    print(path.as_posix())
    # load variable into xarray and calculate daily min.
    ds = xarray.open_dataset(path).load()
    dailyDO = reshape_fvcom3D(
        ds[model_var]
    )
    ds.close()
    # calculate daily minimum
    dailyDO_tmin = calc_fvcom_stat(dailyDO, 'min', axis=1)
    # store time series of minimum across depth levels & save to file
    xr_minDO=xarray.DataArray(dailyDO_tmin) 
    xr_minDO.to_netcdf(output_dir/f'dailyDO_24hrmin_{run_type}.nc')
    
    # store time series of daily min bottom DO & save to file
    dailyDO_tmin_bottom = dailyDO_tmin[:,-1,:]
    xr_minbotDO=xarray.DataArray(dailyDO_tmin_bottom)
    xr_minbotDO.to_netcdf(output_dir/'bottom'/f'dailyDO_24hrmin_bottom_{run_type}.nc')

/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/wqm_baseline/ssm_output.nc


In [ ]:
8784/24